# [Machine Learning with CoreML](https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-core-ml)
**By:** Joshua Newnham (Author)  
**Publisher:** [Packt Publishing](https://www.packtpub.com/) 

## Chapter 7 - Fast Neural Style Transfer 
In this notebook we will look at converting our Fast Neural Style Transfer keras model to Core ML. 

**NB: To run locally;** create the environment from the *coreml27_environment.yml* file in this directory. Details of how to do this can be found [here](https://conda.io/docs/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file). 

In [ ]:
import helpers 
reload(helpers)

First we build the Fast Neural Style Transfer Model passing in the style image used in training the image.  
**Note: This may take sometime as it has to download the VGG weights** 

<img src="images/Van_Gogh-Starry_Night.jpg" width="400px" />

In [ ]:
model = helpers.build_model('images/Van_Gogh-Starry_Night.jpg')

Next we will load our weights; these are the weights that will transform our content image into a stylised version 

In [3]:
model.load_weights('data/van-gogh-starry-night_style.h5')

Let's now print out the architecture of the model (/network); understanding the details in it's entirety is out of scope for this book but it's please take time to scan through it noting down units of of type Lamdba e.g. *res_crop_1 (Lambda)*,  

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 320, 320, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 400, 400, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 400, 400, 64)  15616       zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 400, 400, 64)  256         conv2d_1[0][0]                   
___________________________________________________________________________________________

The reason for identifying these layers is that they are **not** supported in the standard set of models available in **CoreML Tools**. We have a couple of options of supporting them, such as extending modifying the CoreML Tools themselves or creating custom layers. In context of this chapter we are interested in the later (custom layers). So let's inspect each of the layers.  

--- 

### res_crop
The first one we will look at is res_crop; part of the ResNet block that crops (as the name suggests) the output. The code is simple enough; let's have a look at it to see what it does (*nb; this is the function assigned to the Lamdba for the res_crop layer*). 

Essentially all this is doing is cropping the outputs with a padding of 2 on for the width and height axis. We can further interogate this my inspecting the input and output shapes of the custom layer.  

In [5]:
res_crop_3_layer = [layer for layer in model.layers if layer.name == 'res_crop_3'][0] 

print("res_crop_3_layer input shape {}, output shape {}".format(
    res_crop_3_layer.input_shape, res_crop_3_layer.output_shape))

res_crop_3_layer input shape (None, 88, 88, 64), output shape (None, 84, 84, 64)


### rescale_output

Our next custom layer is used at the end of the network to rescale the outputs from the Convolution 2D layer that passes our data through a tanh activation. We can get a better feel what this layer does by inspecting the implementation. 

The method performs an element-wise operation that adds one and scales it by 127.5 (the result of the *tanh* activaiton is that these values would be in a range of -1.0 - 1.0). Similar to above; let's inspect the input and output of this layer. 

In [6]:
rescale_output_layer = [layer for layer in model.layers if layer.name == 'rescale_output'][0]

print("rescale_output_layer input shape {}, output shape {}".format(
    rescale_output_layer.input_shape, 
    rescale_output_layer.output_shape))

rescale_output_layer input shape (None, 320, 320, 3), output shape (None, 320, 320, 3)


## Converting Trained Models to Core ML 

With our model now create and weights loaded; it's time to convert it to a CoreML model; first we need to ensure that the CoreMLTools are install. 

In [7]:
!pip install coremltools

You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    
    import coremltools
    from coremltools.proto import NeuralNetwork_pb2, FeatureTypes_pb2

Let's see what happens when we try to convert the model without acknowledging the custom layers.

In [8]:
coreml_model = coremltools.converters.keras.convert(
    model, 
    input_names=['image'], 
    image_input_names=['image'], 
    output_names="output")

ValueError: Keras layer '<class 'keras.layers.core.Lambda'>' not supported. 

As expected; the method failed returning the error ValueError: Keras layer '' not supported.; let's now proceed to adding support for custom layers.

## Custom Layers 

Next we create a function that will be passed to the **convert** method that will be used to create the layer parameters for our custom layers. Set set the property *className* to the custom class we will implement in Swift along describing it via the *description* property which will come through when inspecting the CoreML model. 

In [9]:
def convert_lambda(layer):
    if layer.function.__name__ == 'rescale_output':
        params = NeuralNetwork_pb2.CustomLayerParams()

        # The name of the Swift or Obj-C class that implements this layer.
        params.className = "RescaleOutputLambda"

        # The desciption is shown in Xcode's mlmodel viewer.
        params.description = "Rescale output to adjust for ReLu ((x+1)*127.5)"

        return params
    elif layer.function.__name__ == 'res_crop':
        params = NeuralNetwork_pb2.CustomLayerParams()

        # The name of the Swift or Obj-C class that implements this layer.
        params.className = "ResCropBlockLambda"

        # The desciption is shown in Xcode's mlmodel viewer.
        params.description = "return x[:, 2:-2, 2:-2]"

        return params
    else:
        raise Exception('Unknown layer')
    return None

It's now time to convert our model; we set the parameter *add_custom_layers* to True so that the conversion process delegates unknown method to our *convert_lambda* function we defined above which we set via the parameter *custom_conversion_functions* which takes a dictionary where the key defines the custom layer name along with a reference to the function to handle creating the customer layer parameters.  

In [10]:
coreml_model = coremltools.converters.keras.convert(
    model, 
    input_names=['image'], 
    image_input_names=['image'], 
    output_names="output",
    add_custom_layers=True,
    custom_conversion_functions={ "Lambda": convert_lambda })

0 : input_1, <keras.engine.topology.InputLayer object at 0x111658890>
1 : zero_padding2d_1, <keras.layers.convolutional.ZeroPadding2D object at 0x1115a8d90>
2 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x104d42150>
3 : batch_normalization_1, <keras.layers.normalization.BatchNormalization object at 0x112a87f10>
4 : activation_1, <keras.layers.core.Activation object at 0x112b43890>
5 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x112a71250>
6 : batch_normalization_2, <keras.layers.normalization.BatchNormalization object at 0x112ad2050>
7 : activation_2, <keras.layers.core.Activation object at 0x1115a8ed0>
8 : conv2d_3, <keras.layers.convolutional.Conv2D object at 0x112c75110>
9 : batch_normalization_3, <keras.layers.normalization.BatchNormalization object at 0x112bdad90>
10 : activation_3, <keras.layers.core.Activation object at 0x112c65890>
11 : conv2d_4, <keras.layers.convolutional.Conv2D object at 0x112d9a890>
12 : batch_normalization_4, <keras.layers.normali

Next we add metadata to our model to help remind us (and anyone else who might use our model) the details of the mode. 

In [12]:
coreml_model.author = 'Joshua Newnham'
coreml_model.license = 'BSD'
coreml_model.short_description = 'FastStyle Transfer with style Van Gogh Starry Night'
coreml_model.input_description['image'] = 'Preprocessed content image'
coreml_model.output_description['output'] = 'Stylized content image'

## Modifying the format of the output 

We could stop here but if you were to save the model and import into XCode you will notice that the output is *MultiArrayType* and we would need to manually convert it to an image in Swift; it would be more convieient to have our CoreML model output a image we can simply use. In order to do this we will modify the output of our model. 

In [13]:
# First we get the spec of our model (the definition of the model that is used by 
# XCode when importing and serialising the model)
spec = coreml_model.get_spec() 

In [14]:
# Next we get reference to the output from the specification (this is what we will be modifying)
output = [output for output in spec.description.output if output.name == 'output'][0]

In [15]:
print(output)

name: "output"
shortDescription: "Stylized content image"
type {
  multiArrayType {
    shape: 3
    shape: 320
    shape: 320
    dataType: DOUBLE
  }
}



In [28]:
# Next check if we need to convert it 
if output.type.WhichOneof('Type') == 'multiArrayType':  
    # Get the shape of the output 
    array_shape = tuple(output.type.multiArrayType.shape)  
    channels, height, width = array_shape   # NB; we don't use channels here but you could generalise 
                                            # this to handle grayscale images 
    
    # Next we set the imageType properties to indicate that we are expecting a image for this output 
    
    # Update the colorspace of the output     
    output.type.imageType.colorSpace = FeatureTypes_pb2.ImageFeatureType.ColorSpace.Value('RGB')  
    
    # Update the image dimensions 
    output.type.imageType.width = width  
    output.type.imageType.height = height
    
    # Recreate the coreml using the new specs 
    coreml_model = coremltools.models.MLModel(spec)  

In [ ]:
# Let's, again, get reference to the output from the specification to check inspect it's type 
output = [output for output in spec.description.output if output.name == 'output'][0]
print(output)

## Save Core ML model 

In [30]:
# Finall we can save it 
coreml_model.save('output/FastStyleTransferVanGoghStarryNight.mlmodel')

## References
- [Converting Trained Models to Core ML](https://developer.apple.com/documentation/coreml/converting_trained_models_to_core_ml)
- [CoreML Tools Documentation](https://apple.github.io/coremltools/index.html)